<a href="https://colab.research.google.com/github/shweetak/Sweta/blob/main/Crime_Data_Analysis_spark_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install Apache Spark 3.0.1 with Hadoop 2.7 from here.
!wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz

# Now, we just need to unzip that folder.
!tar -xvzf spark-3.0.0-bin-hadoop2.7.tgz
!pip install findspark


import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
import findspark
findspark.init()


In [3]:
# 1. Load the covid 19 data in to a spark dataframe (country_wise_latest.csv) with the correct schema definition.

from pyspark.sql import SparkSession
from pyspark.sql.types import *
spark = SparkSession.builder.master("local").appName("Crime_Data_Analysis").getOrCreate()

df = spark.read.csv(path = "/crime_incidents_2013.csv",header=True)
df.show()


+--------+--------------------+--------+-----------+------+--------------------+--------------------+-----------------+-----------------+----+---+--------+---+-------------------+---------------------------+-----------+------------+---------------+--------------------+--------------------+
|     CCN|      REPORTDATETIME|   SHIFT|    OFFENSE|METHOD|    LASTMODIFIEDDATE|    BLOCKSITEADDRESS|      BLOCKXCOORD|      BLOCKYCOORD|WARD|ANC|DISTRICT|PSA|NEIGHBORHOODCLUSTER|BUSINESSIMPROVEMENTDISTRICT|BLOCK_GROUP|CENSUS_TRACT|VOTING_PRECINCT|          START_DATE|            END_DATE|
+--------+--------------------+--------+-----------+------+--------------------+--------------------+-----------------+-----------------+----+---+--------+---+-------------------+---------------------------+-----------+------------+---------------+--------------------+--------------------+
|04104147|4/16/2013 12:00:0...|MIDNIGHT|   HOMICIDE| KNIFE|5/15/2015 8:33:04 AM|1500 - 1599 BLOCK...|           398943|        

In [4]:
# 1. How many offenses were there in 2013?

offenses_count_2013 = df.select("OFFENSE").distinct().count()
print(f"Number of offenses in 2013: {offenses_count_2013}")

Number of offenses in 2013: 9


In [6]:
# 2. How many crimes were committed in each offense

crimes_per_offense = df.groupBy("OFFENSE").count()
print("Crimes committed in each offense:")
crimes_per_offense.show(truncate=False)

Crimes committed in each offense:
+--------------------------+-----+
|OFFENSE                   |count|
+--------------------------+-----+
|ASSAULT W/DANGEROUS WEAPON|2311 |
|THEFT F/AUTO              |10134|
|ARSON                     |35   |
|MOTOR VEHICLE THEFT       |2677 |
|ROBBERY                   |4072 |
|HOMICIDE                  |104  |
|SEX ABUSE                 |299  |
|THEFT/OTHER               |12894|
|BURGLARY                  |3369 |
+--------------------------+-----+



In [7]:
# 3. How many different methods were used (And their count) in offense “Homicide"

homicide_methods_count = (df.filter(df["OFFENSE"] == "HOMICIDE").groupBy("METHOD").count())
print("3. Different methods used in offense 'Homicide' and their count:")
homicide_methods_count.show(truncate=False)


3. Different methods used in offense 'Homicide' and their count:
+------+-----+
|METHOD|count|
+------+-----+
|OTHERS|12   |
|KNIFE |11   |
|GUN   |81   |
+------+-----+



In [9]:
# 4. Which shift had the maximum crimes?

shift_max_crimes = (df.groupBy("SHIFT").count().orderBy("count", ascending=False).first())
max_crimes_shift = shift_max_crimes["SHIFT"]
print(f"Shift with the maximum crimes: {max_crimes_shift}")


Shift with the maximum crimes: EVENING
